In [2]:
from langchain_google_vertexai import ChatVertexAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate
)

# from langchain.output_parsers import JsonOutputParser

import os
import json
import re

from typing import Set

import random

## Загрузка датасета

In [3]:
PATH = "..\\src\\data\\komus\\dataset.json"

In [4]:
with open(PATH, "r", encoding="UTF-8") as file:
    data = json.load(file)

In [5]:
print(len(data))

83755


## Определение параметров

In [6]:
PROJECT_ID = 'axial-chemist-425510-p2'
LOCATION = "europe-west4"

# Укажите путь к вашему JSON-файлу с кредами
SERVICE_ACCOUNT_FILE = "../secrets/axial-chemist-425510-p2-b4eb1d622fbe.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = SERVICE_ACCOUNT_FILE


llm_params = {
    'model_name': 'gemini-2.0-flash-001',
    'project': PROJECT_ID,
    'location': LOCATION
}

llm = ChatVertexAI(**llm_params)

In [ ]:
product = data[41412]

PROMPT = f"""
Ты экспертная система по определению соответствия товарной позиции и ее разложения.
Пример работы:
1) Товарная позиция: Карандаш чернографитный пластиковый НВ с ластиком Выбор есть заточенный шестигранный черный корпус (12 штук в наборе).
Разложение: "Торговая марка": "Выбор есть", "Твердость грифеля": "HB", "Наличие ластика": "Да", "Заточенный": "Да", "Профиль карандаша": "шестигранный", "Материал корпуса": "пластик", "Цвет корпуса": "черный", "Длина корпуса карандаша": "185 мм", "Единица продажи": "упаковка", "Кол-во штук в единице продажи": "12", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "6", "Количество единиц продаж в транспортной упаковке": "90", "Страна происхождения": "Россия"
Ответ: ["Торговая марка", "Наличие ластика", "Твердость грифеля", "Заточенный", "Профиль карандаша", "Цвет корпуса", "Кол-во штук в единице продажи"]
2) Товарная позиция: Сабо женские EVART Ола ЭВА серые (размер 39).
Разложение: "Торговая марка": "Evart", "Коллекция": "Сабо ЭВА Ола", "Тип": "сабо", "Модель": "Ола", "Материал верха обуви": "ЭВА", "Пол": "женский", "Размер": "39", "Цвет": "серый", "Материал подошвы": "ЭВА", "Материал стельки": "отсутствует", "Стелька": "отсутствует", "Подносок": "нет", "Защитные свойства": "от общих загрязнений", "Метод крепления подошвы": "литьевой", "С ремешком": "да", "Перфорация": "Да", "Тип подошвы": "плоская", "Сфера применения": "медицина , пищевое производство и общественное питание , хорека", "Стандарты": "ТР ТС 017/2011", "Честный знак": "Да", "Количество единиц продаж в транспортной упаковке": "1", "Страна происхождения": "Россия", "Вес": "0.227 кг"
Ответ: ["Торговая марка", "Тип", "Модель", "Материал верха обуви", "Пол", "Размер", "Цвет", "Материал подошвы"]
3) Товарная позиция: Бумага для цветной лазерной печати Xerox Colotech + ( A4, 250 г/кв.м, 250 листов, 003R94671)
Разложение: "Торговая марка": "Xerox Colotech +", "Покрытие": "без покрытия (каландрированная бумага)", "Плотность листа бумаги": "250 г/кв.м", "Белизна CIE": "161 ± 3%", "Формат листов": "А4", "Сертификация по экологическим стандартам": "Нет", "Яркость бумаги": "110", "Количество единиц продаж в транспортной упаковке": "144", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "4", "Страна происхождения": "Австрия", "Количество листов в упаковке": "250 шт"
Ответ: ["Торговая марка", "Плотность листа бумаги", "Формат листов", "Количество листов в упаковке"]

Дай ответ на данную позицию {product["title"]} и ее разложение {json.dumps(product["attributes"], ensure_ascii=False)}
""".strip()
PROMPT

'Ты экспертная система по определению соответствия товарной позиции и ее разложения.\nПример работы:\n1) Товарная позиция: Карандаш чернографитный пластиковый НВ с ластиком Выбор есть заточенный шестигранный черный корпус (12 штук в наборе).\nРазложение: "Торговая марка": "Выбор есть", "Твердость грифеля": "HB", "Наличие ластика": "Да", "Заточенный": "Да", "Профиль карандаша": "шестигранный", "Материал корпуса": "пластик", "Цвет корпуса": "черный", "Длина корпуса карандаша": "185 мм", "Единица продажи": "упаковка", "Кол-во штук в единице продажи": "12", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "6", "Количество единиц продаж в транспортной упаковке": "90", "Страна происхождения": "Россия"\nОтвет: ["Торговая марка", "Наличие ластика", "Твердость грифеля", "Заточенный", "Профиль карандаша", "Цвет корпуса", "Кол-во штук в единице продажи"]\n2) Товарная позиция: Сабо женские EVART Ола ЭВА серые (размер 39).\nРазложение: "Торговая марка": "Evart", "Коллекция": "Сабо ЭВА О

In [38]:
res = llm.invoke(PROMPT)

res_text = res.content

res_text

'["Торговая марка", "Вид товара", "Материал", "Объем полный", "Объем рабочий", "Цвет", "Кол-во штук в единице продажи"]\n'

## Примеры для Few-Shot

In [68]:
examples = [
    {
        "title": "Карандаш чернографитный пластиковый НВ с ластиком Выбор есть заточенный шестигранный черный корпус (12 штук в наборе)",
        "attributes": '{ "Торговая марка": "Выбор есть", "Твердость грифеля": "HB", "Наличие ластика": "Да", "Заточенный": "Да", "Профиль карандаша": "шестигранный", "Материал корпуса": "пластик", "Цвет корпуса": "черный", "Длина корпуса карандаша": "185 мм", "Единица продажи": "упаковка", "Кол-во штук в единице продажи": "12", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "6", "Количество единиц продаж в транспортной упаковке": "90", "Страна происхождения": "Россия" }',
        "result": '["Торговая марка", "Наличие ластика", "Твердость грифеля", "Заточенный", "Профиль карандаша", "Цвет корпуса", "Кол-во штук в единице продажи"]'
    },
    {
        "title": "Сабо женские EVART Ола ЭВА серые (размер 39)",
        "attributes": '{"Торговая марка": "Evart", "Коллекция": "Сабо ЭВА Ола", "Тип": "сабо", "Модель": "Ола", "Материал верха обуви": "ЭВА", "Пол": "женский", "Размер": "39", "Цвет": "серый", "Материал подошвы": "ЭВА", "Материал стельки": "отсутствует", "Стелька": "отсутствует", "Подносок": "нет", "Защитные свойства": "от общих загрязнений", "Метод крепления подошвы": "литьевой", "С ремешком": "да", "Перфорация": "Да", "Тип подошвы": "плоская", "Сфера применения": "медицина , пищевое производство и общественное питание , хорека", "Стандарты": "ТР ТС 017/2011", "Честный знак": "Да", "Количество единиц продаж в транспортной упаковке": "1", "Страна происхождения": "Россия", "Вес": "0.227 кг"}',
        "result": '["Торговая марка", "Тип", "Модель", "Материал верха обуви", "Пол", "Размер", "Цвет", "Материал подошвы"]'
    },
    {
        "title": "Бумага для цветной лазерной печати Xerox Colotech + ( A4, 250 г/кв.м, 250 листов, 003R94671)",
        "attributes": '{"Торговая марка": "Xerox Colotech +", "Покрытие": "без покрытия (каландрированная бумага)", "Плотность листа бумаги": "250 г/кв.м", "Белизна CIE": "161 ± 3%", "Формат листов": "А4", "Сертификация по экологическим стандартам": "Нет", "Яркость бумаги": "110", "Количество единиц продаж в транспортной упаковке": "144", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "4", "Страна происхождения": "Австрия", "Количество листов в упаковке": "250 шт"}',
        "result": '["Торговая марка", "Плотность листа бумаги", "Формат листов", "Количество листов в упаковке"]'
    }
]

examples_str = "\n".join([f"""
{i+1})Title: {product["title"]}
Decomposition: {product['attributes']}
Example of your: {product['result']} 
""".strip() for i, product in enumerate(examples)])
print(examples_str)

1)Title: Карандаш чернографитный пластиковый НВ с ластиком Выбор есть заточенный шестигранный черный корпус (12 штук в наборе)
Decomposition: { "Торговая марка": "Выбор есть", "Твердость грифеля": "HB", "Наличие ластика": "Да", "Заточенный": "Да", "Профиль карандаша": "шестигранный", "Материал корпуса": "пластик", "Цвет корпуса": "черный", "Длина корпуса карандаша": "185 мм", "Единица продажи": "упаковка", "Кол-во штук в единице продажи": "12", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "6", "Количество единиц продаж в транспортной упаковке": "90", "Страна происхождения": "Россия" }
Example of your: ["Торговая марка", "Наличие ластика", "Твердость грифеля", "Заточенный", "Профиль карандаша", "Цвет корпуса", "Кол-во штук в единице продажи"]
2)Title: Сабо женские EVART Ола ЭВА серые (размер 39)
Decomposition: {"Торговая марка": "Evart", "Коллекция": "Сабо ЭВА Ола", "Тип": "сабо", "Модель": "Ола", "Материал верха обуви": "ЭВА", "Пол": "женский", "Размер": "39", "Цвет": "

## Создание промптов

In [ ]:
SYSTEM_PROMPT = """
You are an expert system for determining the conformity of a product item and its decomposition.
Example of works: 
{examples}

The output should be in the form of an array.

Ensure that:
- Write only the keys of the attributes whose values ​​are present in the decomposition.
- Do not write keys values ​​that are not present in the title

**Do not generate any text after the JSON output.**
""".strip()

USER_PROMPT = """Give an answer to this position {problem_title} and its decomposition {problem_decomposition}.""".strip()

In [57]:
def create_model_prompts(system_prompt: str,
                         user_prompt: str) -> ChatPromptTemplate:
    system_prompt = SystemMessagePromptTemplate.from_template(system_prompt)
    user_prompt = HumanMessagePromptTemplate.from_template(user_prompt)
    chat_prompt = ChatPromptTemplate.from_messages(
        [system_prompt,
         user_prompt]
    )
    return chat_prompt


prompt = create_model_prompts(SYSTEM_PROMPT, USER_PROMPT)



## Случайный выбор из подборки

In [69]:
idx = random.choices(list(range(len(data))), k = 10)
idx

[10446, 15843, 70525, 861, 2381, 65072, 53537, 65497, 35815, 21097]

## Запуск модели

In [70]:
similarity_chain = prompt | llm

for id in idx:
    product = data[id]

    res = similarity_chain.invoke({
        "examples": "examples_str",
        "problem_title": product["title"],
        "problem_decomposition": json.dumps(product["attributes"], ensure_ascii=False)
    })

    print(product["title"])
    print(json.dumps(product["attributes"], ensure_ascii=False, indent=4))
    print(f"Ответ: {res.content}")
    

Царга к столу Onix O.M-CS-4 с креплением (белый бриллиант/белый, 1450х18х320 мм)
{
    "Коллекция": "Мебель для персонала ONIX-П",
    "Тип": "царга для стола",
    "Цвет покрытия": "белый , белый бриллиант",
    "Материал": "ЛДСП",
    "Место расположения": "фронтальное",
    "Крепеж в комплекте": "Да",
    "Сборка мебели": "бесплатная сборка",
    "Гарантийный срок": "18 мес",
    "Страна происхождения": "Россия",
    "Высота": "320 мм",
    "Ширина": "1450 мм",
    "Глубина": "18 мм"
}
Ответ: ```json
{
"Коллекция": "Мебель для персонала ONIX-П",
"Тип": "царга для стола",
"Цвет покрытия": "белый , белый бриллиант",
"Материал": "ЛДСП",
"Крепеж в комплекте": "Да",
"Страна происхождения": "Россия",
"Высота": "320 мм",
"Ширина": "1450 мм",
"Глубина": "18 мм"
}
```
Термогигрометр электронный RGK TH-20 с поверкой (778619)
{
    "Торговая марка": "RGK",
    "Тип": "термогигрометр",
    "ЖК-дисплей": "Да",
    "Память": "нет",
    "Тип питания": "батарейки",
    "Тип измерения": "температура